In [1]:
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

In [2]:
klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

print(klue_mrc_dataset)
print(sentence_model)

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})
SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [3]:
klue_mrc_dataset = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)
print(klue_mrc_dataset)

klue_mrc_dataset = klue_mrc_dataset['train']

embeddings = sentence_model.encode(klue_mrc_dataset['context'])
print(embeddings.shape)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 16554
    })
})
(1000, 768)


In [4]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [5]:
query = "이번 연도에는 언제 비가 많이 올까?"
query_embedding = sentence_model.encode([query])

distances, indices = index.search(query_embedding, 3)
print(distances)
print(indices)

[[300.91888 420.86273 420.86273]]
[[  0 920 921]]


In [6]:
for idx in indices[0]:
    print(klue_mrc_dataset['context'][idx])

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 중에서도 태평양먹장어(Eptatretus stoutii)와 구조가 비슷하다. 게다가 오리너구리의 눈에서는 같은 포유류에게서 찾아보기 힘든 이중 원추세포가 발견된다. 

크게 발달하지도 않았고 물 속에서도 시야 확보를 위하여 쓰이지 않는 눈이지만, 오리너구리의 조상뻘 되는 종들이 한때 시각에 보다 크게 의존했다는 증거는 많이 남아 있다. 오리너구리의 눈 구조를 살펴보면 수정체와 그와 접하는 각

In [7]:
query = klue_mrc_dataset[3]['question']
print(query, '\n')

query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
    print("=" * 50)
    print(idx)
    print(klue_mrc_dataset['context'][idx])

로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가? 

78
태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기로 결정했다. 그 첫 단계로 필리핀 방면의 전략 거점의 확보가 필요하였으며, 뉴기니 서쪽에 위치한 말루쿠 제도의 모로타이 섬을 공격 목표로 정했다. 또한 동시에 팔라우 제도의 펠렐리우 섬과 앙가우르 섬에도 미국 해군 주도의 연합국 중부 태평양 방면군이 공략을 맡았다.(이때의 전략 결정의 경위에 대해서는 필리핀 전투 (1944 - 1945)#미국을 참조.)

한편, 1942년에 네덜란드령 동인도의 일부였던 모로타이 섬을 점령한 일본군은 이후 수비 부대를 증강배치하지 않았다. 1944년 말루쿠 제도 방면의 방비 강화를 도모하고자 파견된 제32사단은 평야가 많은 비행장 건설에 적합한 주변의 할마헤라 섬을 방어의 중심으로 여겼다. 따라서 모로타이 섬에는 제32사단의 2개 대대가 비행장 건설을 추진했지만, 배수가 좋지 않아 건설을 포기했다. 이 2개 대대가 할마헤라 섬으로 철수한 이후에는 카와시마 타케노부(川島威伸) 중위를 지휘관으로 하는 제2유격대 소속의 2개 중대(주로 다카사고의용대)만 배치되어 있었다.

연합군이 상륙했을 때, 섬에는 9000명의 현지인이 살고 있었다. 도민에 대한 선무공작을 수행하기 위해 연합군의 상륙 부대에는 네덜란드 군 민정반이 추가 되었다.
79
태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기로 결정했다. 그 첫 단계로 필리핀 방면의 전략 거점의 확보가 필요하였으며, 뉴기니 서쪽에 위치한 말루쿠 제도의 모로타이 섬을 공격 목표로 정했다. 또한 동시에 팔라우 제도의 펠렐리우 섬과 앙가우르 섬에도 미국 해군 주도의 연합국 중부 태평양 방면군이 공략을 맡았다

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document, VectorStoreIndex, ServiceContext, Settings

In [11]:
# embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large-instruct")
# service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

Settings.embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large-instruct")
Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [12]:
text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index_llama = VectorStoreIndex.from_documents(documents)

In [13]:
import math
import numpy as np

from typing import List
from collections import defaultdict
from transformers import PreTrainedTokenizer

In [14]:
class BM25:
    def __init__(self, corpus, tokenizer):
        """
        tokenizer : 토크나이저
        corpus : 문서 데이터셋
        """
        self.tokenizer = tokenizer
        self.corpus = corpus

        ## 각 문서를 토큰화
        self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
        
        ## 토큰화된 문서들의 수
        self.n_docs = len(self.tokenized_corpus)

        ## 전체 문서들의 평균 길이
        self.avg_doc_lens = sum(len(lst) for lst in self.tokenized_corpus) / len(self.tokenized_corpus)

        ## IDF 계산
        self.idf = self.calculate_idf()

        ## TF 계산
        self.term_freqs = self.calculate_term_freqs()

    def calculate_idf(self):
        idf = defaultdict(float)
        ## 토큰화된 문서들을 순회
        for doc in self.tokenized_corpus:
            ## 토큰화된 문서의 토큰들의 등장 횟수를 카운팅하고 딕셔너리에 저장.
            for token_id in set(doc):
                idf[token_id] += 1

        for token_id, doc_frequency in idf.items():
            idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)

        return idf
    
    def calculate_term_freqs(self):
        term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
        
        ## 토큰화된 문서들을 순회
        for i, doc in enumerate(self.tokenized_corpus):
            for token_id in doc:
                term_freqs[i][token_id] += 1

        return term_freqs
    
    def get_scores(self, query, k1=1.2, b=0.75):
        query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
        scores = np.zeros(self.n_docs)

        for q in query:
            idf = self.idf[q]
            for i, term_freq in enumerate(self.term_freqs):
                q_frequency = term_freq[q]
                doc_len = len(self.tokenized_corpus[i])
                score_q = idf * (q_frequency * (k1 + 1)) / ((q_frequency) + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
                scores[i] += score_q
        
        return scores
    
    def get_top_k(self, query, k):
        scores = self.get_scores(query)
        top_k_indices = np.argsort(scores)[-k:][::-1]
        top_k_scores = scores[top_k_indices]

        return top_k_scores, top_k_indices

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

bm25 = BM25(['안녕하세요', '반갑습니다', '안녕 서울'], tokenizer)
bm25.get_scores('안녕')

array([0.44713859, 0.        , 0.52354835])

In [18]:
bm25 = BM25(klue_mrc_dataset['context'], tokenizer)

query = '아번 연도에는 언제 비가 많이 올까?'
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
    print(klue_mrc_dataset['context'][idx][:50])

갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다
베토벤의 후기는 1810-1819년에 시작되었다. 베토벤은 그의 청력 저하로 의기소침해 있
도비는 감독이 되는 데 자신의 전망을 가졌다. 겨울 야구를 감독한 시즌 후에 그는 자신에게


In [19]:
query = klue_mrc_dataset[3]['question']
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
    print(klue_mrc_dataset['context'][idx][:50])

미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
;메카동(メカドン)
:성우 : 나라하시 미키(ならはしみき)
길가에 버려져 있던 낡은 느티나
;메카동(メカドン)
:성우 : 나라하시 미키(ならはしみき)
길가에 버려져 있던 낡은 느티나


In [20]:
from collections import defaultdict

def reciprocal_rank_fusion(rankings, k=5):
    rrf = defaultdict(float)
    for ranking in rankings:
        for i, doc_id in enumerate(ranking, 1):
            rrf[doc_id] += 1.0 / (k + i)

    return sorted(rrf.items(), key=lambda x : x[1], reverse=True)

In [21]:
rankings = [[1, 4, 3, 5, 6], [2, 1, 3, 6, 4]]
reciprocal_rank_fusion(rankings)

[(1, 0.30952380952380953),
 (3, 0.25),
 (4, 0.24285714285714285),
 (6, 0.2111111111111111),
 (2, 0.16666666666666666),
 (5, 0.1111111111111111)]

In [22]:
def dense_vector_search(query, k):
    query_embedding = sentence_model.encode([query])
    distances, indices = index.search(query_embedding, k)
    
    return distances[0], indices[0]

def hybrid_search(query, k=20):
    _, dense_search_ranking = dense_vector_search(query, 100)
    _, bm25_search_ranking = bm25.get_top_k(query, 100)

    results = reciprocal_rank_fusion([dense_search_ranking, bm25_search_ranking], k=k)

    return results

In [23]:
query = "이번 연도에는 언제 비가 많이 올까?"
print("검색 쿼리 문장: ", query)
results = hybrid_search(query)
for idx, score in results[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

print("=" * 80)
query = klue_mrc_dataset[3]['question'] # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
print("검색 쿼리 문장: ", query)

results = hybrid_search(query)
for idx, score in results[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

검색 쿼리 문장:  이번 연도에는 언제 비가 많이 올까?
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
검색 쿼리 문장:  로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음
1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음
